In [ ]:
# Data loading (features - springs)

import pandas as pd
import json

json_features_file = open("flickr-spring-features.json")

parsed_json_features = json.load(json_features_file)

df_features = pd.json_normalize(parsed_json_features["features"])

df_features = df_features[["latLng", "rank", "resolutions.ALL.fnRank", "resolutions.ALL.sigRank", "resolutions.ALL.maxRank"]]
df_features = df_features.rename(columns={"resolutions.ALL.fnRank": "fnRank", "resolutions.ALL.sigRank": "sigRank", "resolutions.ALL.maxRank": "maxRank"})

df_features = df_features.dropna(subset=["fnRank"])
df_features = df_features.dropna(subset=["sigRank"])
df_features = df_features.dropna(subset=["maxRank"])

return df_features

Index(['rank', 'fnRank', 'sigRank'], dtype='object')


In [ ]:
# Computation Analysis (computing rank)

import numpy as np

df_scatterplot = arg[["rank", "fnRank", "sigRank", "maxRank"]]

df_scatterplot["combinedRanks"] = np.sqrt(
    df_scatterplot["maxRank"] ** 2 +
    df_scatterplot["fnRank"] ** 2 +
    df_scatterplot["sigRank"] ** 2
)

return df_scatterplot

In [ ]:
# Vega-Lite

{ 
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json", 
  "title": "RANK (spring)",
  "params": [ {"name": "clickSelect", "select": "interval"} ], 
  "mark": { "type": "point", "cursor": "pointer" }, 
  "encoding": { 
    "x": {"field": "combinedRanks", "type": "quantitative"},
    "y": {"field": "rank", "type": "quantitative"}, 
    "fillOpacity": { 
        "condition": {"param": "clickSelect", "value": 1}, 
        "value": 0.3 
    }, 
    "color": { 
      "field": "interacted", 
      "type": "nominal", 
      "condition": {
        "test": "datum.interacted === '1'", "value": "red", "else": "blue"} } 
  }, 
  "config": { "scale": { "bandPaddingInner": 0.2 } } 
} 



In [ ]:
# Data transformation (connected to df_features - convert latLng into points and create buffer)

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

df_points = arg

df_points = df_points.explode("latLng", ignore_index=True)

df_points["geometry"] = df_points["latLng"].apply(lambda x: Point(x[1], x[0]))  # (lon, lat)

gdf_points = gpd.GeoDataFrame(df_points, geometry="geometry", crs="EPSG:4326")

gdf_points = gdf_points.drop(columns=["latLng"])

gdf_points["value"] = 1

gdf_points = gdf_points.to_crs(3857)

gdf_points["geometry"] = gdf_points["geometry"].buffer(25)

gdf_points = gdf_points[["geometry", "value"]]

gdf_points = gdf_points.to_crs(3395)

gdf_points.metadata = {
    'name': 'pulse'
}

return gdf_points

In [ ]:
# Data load (load background map for Manhattan)

import utk

uc = utk.OSM.load([40.721080386107815, -73.99495342796531, 40.76282380707685, -73.95809371928384], layers=['parks', 'roads'])

#parks
json_parks = uc.layers['json'][0]
gdf_parks = uc.layers['gdf']['objects'][0]
gdf_parks.metadata = {
 'name': 'parks'
}

json_roads = uc.layers['json'][1]
gdf_roads = uc.layers['gdf']['objects'][1]
gdf_roads.metadata = {
 'name': 'roads'
}

return gdf_parks, gdf_roads

In [ ]:
# Merge (gdf_points and gdf_parks, gdf_roads)

In [ ]:
# Computation Analysis

return arg[0], arg[1][0], arg[1][1]

In [ ]:
# Data loading (features - summer)

import pandas as pd
import json
import numpy as np

json_features_file = open("flickr-summer-features.json")

parsed_json_features = json.load(json_features_file)

df_scatterplot = pd.json_normalize(parsed_json_features["features"])

df_scatterplot = df_scatterplot[["rank", "resolutions.ALL.fnRank", "resolutions.ALL.sigRank", "resolutions.ALL.maxRank"]]
df_scatterplot = df_scatterplot.rename(columns={"resolutions.ALL.fnRank": "fnRank", "resolutions.ALL.sigRank": "sigRank", "resolutions.ALL.maxRank": "maxRank"})

df_scatterplot = df_scatterplot.dropna(subset=["fnRank"])
df_scatterplot = df_scatterplot.dropna(subset=["sigRank"])
df_scatterplot = df_scatterplot.dropna(subset=["maxRank"])

df_scatterplot["combinedRanks"] = np.sqrt(
    df_scatterplot["maxRank"] ** 2 +
    df_scatterplot["fnRank"] ** 2 +
    df_scatterplot["sigRank"] ** 2
)

return df_scatterplot

In [ ]:
# Vega-lite

{ 
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json", 
  "title": "RANK (summer)",
  "params": [ {"name": "clickSelect", "select": "interval"} ], 
  "mark": { "type": "point", "cursor": "pointer" }, 
  "encoding": { 
    "x": {"field": "combinedRanks", "type": "quantitative"},
    "y": {"field": "rank", "type": "quantitative"}, 
    "fillOpacity": { 
        "condition": {"param": "clickSelect", "value": 1}, 
        "value": 0.3 
    }, 
    "color": { 
      "field": "interacted", 
      "type": "nominal", 
      "condition": {
        "test": "datum.interacted === '1'", "value": "red", "else": "blue"} } 
  }, 
  "config": { "scale": { "bandPaddingInner": 0.2 } } 
} 

In [ ]:
# Data loading (features - fall)

import pandas as pd
import json
import numpy as np

json_features_file = open("flickr-fall-features.json")

parsed_json_features = json.load(json_features_file)

df_scatterplot = pd.json_normalize(parsed_json_features["features"])

df_scatterplot = df_scatterplot[["rank", "resolutions.ALL.fnRank", "resolutions.ALL.sigRank", "resolutions.ALL.maxRank"]]
df_scatterplot = df_scatterplot.rename(columns={"resolutions.ALL.fnRank": "fnRank", "resolutions.ALL.sigRank": "sigRank", "resolutions.ALL.maxRank": "maxRank"})

df_scatterplot = df_scatterplot.dropna(subset=["fnRank"])
df_scatterplot = df_scatterplot.dropna(subset=["sigRank"])
df_scatterplot = df_scatterplot.dropna(subset=["maxRank"])

df_scatterplot["combinedRanks"] = np.sqrt(
    df_scatterplot["maxRank"] ** 2 +
    df_scatterplot["fnRank"] ** 2 +
    df_scatterplot["sigRank"] ** 2
)

return df_scatterplot

In [ ]:
# Vega-lite

{ 
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json", 
  "title": "RANK (fall)",
  "params": [ {"name": "clickSelect", "select": "interval"} ], 
  "mark": { "type": "point", "cursor": "pointer" }, 
  "encoding": { 
    "x": {"field": "combinedRanks", "type": "quantitative"},
    "y": {"field": "rank", "type": "quantitative"}, 
    "fillOpacity": { 
        "condition": {"param": "clickSelect", "value": 1}, 
        "value": 0.3 
    }, 
    "color": { 
      "field": "interacted", 
      "type": "nominal", 
      "condition": {
        "test": "datum.interacted === '1'", "value": "red", "else": "blue"} } 
  }, 
  "config": { "scale": { "bandPaddingInner": 0.2 } } 
} 

In [ ]:
# Data loading (features - winter)

import pandas as pd
import json
import numpy as np

json_features_file = open("flickr-winter-features.json")

parsed_json_features = json.load(json_features_file)

df_scatterplot = pd.json_normalize(parsed_json_features["features"])

df_scatterplot = df_scatterplot[["rank", "resolutions.ALL.fnRank", "resolutions.ALL.sigRank", "resolutions.ALL.maxRank"]]
df_scatterplot = df_scatterplot.rename(columns={"resolutions.ALL.fnRank": "fnRank", "resolutions.ALL.sigRank": "sigRank", "resolutions.ALL.maxRank": "maxRank"})

df_scatterplot = df_scatterplot.dropna(subset=["fnRank"])
df_scatterplot = df_scatterplot.dropna(subset=["sigRank"])
df_scatterplot = df_scatterplot.dropna(subset=["maxRank"])

df_scatterplot["combinedRanks"] = np.sqrt(
    df_scatterplot["maxRank"] ** 2 +
    df_scatterplot["fnRank"] ** 2 +
    df_scatterplot["sigRank"] ** 2
)

return df_scatterplot

In [ ]:
# Vega-lite

{ 
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json", 
  "title": "RANK (winter)",
  "params": [ {"name": "clickSelect", "select": "interval"} ], 
  "mark": { "type": "point", "cursor": "pointer" }, 
  "encoding": { 
    "x": {"field": "combinedRanks", "type": "quantitative"},
    "y": {"field": "rank", "type": "quantitative"}, 
    "fillOpacity": { 
        "condition": {"param": "clickSelect", "value": 1}, 
        "value": 0.3 
    }, 
    "color": { 
      "field": "interacted", 
      "type": "nominal", 
      "condition": {
        "test": "datum.interacted === '1'", "value": "red", "else": "blue"} } 
  }, 
  "config": { "scale": { "bandPaddingInner": 0.2 } } 
} 